# Web scrapping TripAdvisor-Mexico

Importing necessary libraries:

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
#A list to store the scraped data
states_list = []

Functions to scrapp the cities:

In [19]:
def formula(num:int) -> int:

    return int(num-1)*20

def extract(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}
    
    url = f'https://www.tripadvisor.com.mx/Restaurants-g150768-oa{page}-Mexico.html#LOCATION_LIST'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

def transform(soup) -> list:
    
    divs = soup.find_all('div', class_='geo_wrap')

    for item in divs:
        state =  item.find('a').text
        href =  item.find('a')['href']

        state ={'state': state, 'href': href}
        states_list.append(state)

    return  states_list

def transform2(soup) -> list:
    
    for ultag in soup.find_all('ul', {'class': 'geoList'}):
        for litag in ultag.find_all('li'):
            state= litag.text
            href= litag.find('a')['href']
            state = {'state':state,'href':href}
            states_list.append(state)



In [20]:
transform(extract(formula(1)))

[{'state': 'Restaurantes en Ciudad de México',
  'href': '/Restaurants-g150800-Mexico_City_Central_Mexico_and_Gulf_Coast.html'},
 {'state': 'Restaurantes en Guadalajara',
  'href': '/Restaurants-g150798-Guadalajara_Guadalajara_Metropolitan_Area.html'},
 {'state': 'Restaurantes en Cancún',
  'href': '/Restaurants-g150807-Cancun_Yucatan_Peninsula.html'},
 {'state': 'Restaurantes en Playa del Carmen',
  'href': '/Restaurants-g150812-Playa_del_Carmen_Yucatan_Peninsula.html'},
 {'state': 'Restaurantes en Monterrey',
  'href': '/Restaurants-g150782-Monterrey_Northern_Mexico.html'},
 {'state': 'Restaurantes en Puerto Vallarta',
  'href': '/Restaurants-g150793-Puerto_Vallarta.html'},
 {'state': 'Restaurantes en Santiago de Querétaro',
  'href': '/Restaurants-g479232-Queretaro_City_Central_Mexico_and_Gulf_Coast.html'},
 {'state': 'Restaurantes en Mérida',
  'href': '/Restaurants-g150811-Merida_Yucatan_Peninsula.html'},
 {'state': 'Restaurantes en Puebla',
  'href': '/Restaurants-g152773-Puebla_

In [21]:
# city_list = []
for i in range(2,3):
    transform2(extract(formula(i)))

In [22]:
city_list = pd.DataFrame(states_list)

In [23]:
city_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   40 non-null     object
 1   href    40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [24]:
city_list

,state,href
0,Restaurantes en Ciudad de México,/Restaurants-g150800-Mexico_City_Central_Mexic...
1,Restaurantes en Guadalajara,/Restaurants-g150798-Guadalajara_Guadalajara_M...
2,Restaurantes en Cancún,/Restaurants-g150807-Cancun_Yucatan_Peninsula....
3,Restaurantes en Playa del Carmen,/Restaurants-g150812-Playa_del_Carmen_Yucatan_...
4,Restaurantes en Monterrey,/Restaurants-g150782-Monterrey_Northern_Mexico...
5,Restaurantes en Puerto Vallarta,/Restaurants-g150793-Puerto_Vallarta.html
6,Restaurantes en Santiago de Querétaro,/Restaurants-g479232-Queretaro_City_Central_Me...
7,Restaurantes en Mérida,/Restaurants-g150811-Merida_Yucatan_Peninsula....
8,Restaurantes en Puebla,/Restaurants-g152773-Puebla_Central_Mexico_and...
9,Restaurantes en Zapopan,/Restaurants-g1006488-Zapopan_Guadalajara_Metr...


In [25]:
city_list.to_csv('city_list.csv', index=False)

Functions to scrap the restaurants name and webpage:

In [26]:
def extractR(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}
    
    url = f'https://www.tripadvisor.com.mx{page}'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

restaurants_list = []

def transformR(soup) -> list:
    
    divs = soup.find_all('div', class_='zdCeB Vt o')

    for item in divs:
        if '.' in item.find('a', class_='Lwqic Cj b').text:

            restaurant =  item.find('a', class_='Lwqic Cj b').text.split('.')[1]
            href = item.find('a', class_='Lwqic Cj b')['href']
            

            restaurants ={#'index':index,
                          'restaurant': restaurant, 
                          'href':href}
                        
            restaurants_list.append(restaurants)

    # return  restaurants_list

In [27]:
for href in city_list['href']:
    transformR(extractR(href))

In [28]:
df_restaurants = pd.DataFrame(restaurants_list )

In [29]:
# df_restaurants.index = df_restaurants['index']
# df_restaurants.drop(['index'], axis=1, inplace=True)

In [30]:
df_restaurants

,restaurant,href
0,Balta,/Restaurant_Review-g150800-d19338104-Reviews-B...
1,La Mansion Marriott Reforma,/Restaurant_Review-g150800-d2394477-Reviews-La...
2,Condimento Restaurant,/Restaurant_Review-g150800-d1926008-Reviews-Co...
3,La Distral,/Restaurant_Review-g150800-d12104177-Reviews-L...
4,Sonora Grill Coapa,/Restaurant_Review-g150800-d20157621-Reviews-S...
...,...,...
1196,Dan's Cafe Deluxe,/Restaurant_Review-g153373-d2418201-Reviews-Da...
1197,Benditos Ristorante Pizzeria Italiana,/Restaurant_Review-g153373-d7322957-Reviews-Be...
1198,Kuhl Frozen Yogurt,/Restaurant_Review-g153373-d3710549-Reviews-Ku...
1199,Sativa Puerto,/Restaurant_Review-g153373-d1180559-Reviews-Sa...


Functions to scrap the restaurants info:

In [91]:
def extractRI(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}

    url = f'https://www.tripadvisor.com.mx{page}'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

restaurants_info_list = []

def transformRI(soup) -> list:
    
    divs = soup.find_all('div', class_='lBkqB _T')

    for item in divs:
        calification = item.find('svg', class_='UctUV d H0')['aria-label']
        number_of_reviwes = item.find('span', class_='AfQtZ').text
        print(calification)
        print(number_of_reviwes)
        
        divs2 = soup.find_all('div', class_='xLvvm ui_column is-12-mobile is-4-desktop')
        print(divs2)
        for item2 in divs2:
            try:
                address = item2.find('span', class_='yEWoV').text
                print('address:',address)
            except:
                pass

            try:
                latitude = item2.find('img', class_='w MD _S')
                print('latitude:',latitude)
            except:
                pass

            try:
                web_site = item2.find('a', class_='YnKZo Ci Wc _S C FPPgD')['href']
                print('web_site:',web_site)
            except:
                pass

            

            # restaurants ={#'index':index,
            #               'restaurant': restaurant, 
            #               'href':href}
                        
            # restaurants_info_list.append(restaurants)

    # return  restaurants_list

In [92]:
transformRI(extractRI(df_restaurants['href'][0]))

5.0 de 5 burbujas
464 opiniones
[<div class="xLvvm ui_column is-12-mobile is-4-desktop"><div class="YDAvY R2 F1 e k"><div><h2>Calificaciones y opiniones</h2><div class="QEQvp"><span class="ZDEqb">5.0<!-- --> </span><span class="MFMAn"><span class="ui_bubble_rating bubble_50"></span></span><a class="IcelI" href="#REVIEWS">465<!-- --> opiniones</a></div><div class="cNFlb"><b class=""><span class="">N.º 1</span></b> de 5,262 <a href="/Restaurants-g150800-Mexico_City_Central_Mexico_and_Gulf_Coast.html">Restaurantes en Ciudad de México</a></div></div><div class="KVjOc Z"></div><div><div class="uUbEd b">CALIFICACIONES</div><div><div class="DzMcu"><span class="ui_icon restaurants lANzp"></span><span class="BPsyj">Comida</span><span class="vzATR"><span class="ui_bubble_rating bubble_45"></span></span></div><div class="DzMcu"><span class="ui_icon bell lANzp"></span><span class="BPsyj">Servicio</span><span class="vzATR"><span class="ui_bubble_rating bubble_50"></span></span></div><div class="DzM